In [1]:
with open("./input.txt", "r",encoding="utf-8") as file:
    data = file.read()
    # print(data)
import torch

voc=sorted(list(set(data)))
# print(len(data))
# print(list(set(data)))
# print(set(data))
# print(voc)
chtoi={c:i for i,c in enumerate(voc)}
itoch={i:c for i,c in enumerate(voc)}
encoding=lambda s: [chtoi[c] for c in s]
decoding=lambda s: "".join([itoch[c] for c in s])


# print(encoding("hello world"))
# print(decoding(encoding("hello world")))

datatset=torch.tensor(encoding(data),dtype=torch.long)
# print(datatset[:100])
n=int(0.9*len(datatset))
trainset=datatset[:n]
testset=datatset[n:]
block_size=8
x_train=datatset[:block_size+1]
y_train=datatset[1:block_size+1]
# print(x_train)
for i in range(block_size):
    context=x_train[:i+1]
    target=y_train[i]
    # print(f"context: {context}, target: {target}")
torch.manual_seed(42)
batch_size=4
block_size=8
def getbatch(split):
    data=trainset if split=="train" else testset
    ix=torch.randint(len(data)-block_size,(batch_size,))
    x=torch.stack(([data[i:i+block_size] for i in ix]))
    y=torch.stack(([data[i+1:i+block_size+1] for i in ix]))
    print(ix)
    return x,y
xb,yb=getbatch("train")
print(xb)
print(yb)

for b in range(batch_size):
    for a in range(block_size):
        context=xb[b,:a+1]
        target=yb[b,a]
        print(f"context: {context}, target: {target}")
    


/mnt/c9542373-6611-4b24-a6c7-2979563b4e53/gptdev/gptenv/lib/python3.11/site-packages/torch/_subclasses/functional_tensor.py:275: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


tensor([476250,  18899, 645194, 831150])
tensor([[57,  1, 46, 47, 57,  1, 50, 53],
        [ 1, 58, 46, 43, 56, 43,  1, 41],
        [17, 26, 15, 17, 10,  0, 32, 53],
        [57, 58,  6,  1, 61, 47, 58, 46]])
tensor([[ 1, 46, 47, 57,  1, 50, 53, 60],
        [58, 46, 43, 56, 43,  1, 41, 39],
        [26, 15, 17, 10,  0, 32, 53,  1],
        [58,  6,  1, 61, 47, 58, 46,  0]])
context: tensor([57]), target: 1
context: tensor([57,  1]), target: 46
context: tensor([57,  1, 46]), target: 47
context: tensor([57,  1, 46, 47]), target: 57
context: tensor([57,  1, 46, 47, 57]), target: 1
context: tensor([57,  1, 46, 47, 57,  1]), target: 50
context: tensor([57,  1, 46, 47, 57,  1, 50]), target: 53
context: tensor([57,  1, 46, 47, 57,  1, 50, 53]), target: 60
context: tensor([1]), target: 58
context: tensor([ 1, 58]), target: 46
context: tensor([ 1, 58, 46]), target: 43
context: tensor([ 1, 58, 46, 43]), target: 56
context: tensor([ 1, 58, 46, 43, 56]), target: 43
context: tensor([ 1, 58, 46, 4

In [4]:
import torch 
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(42)


In [8]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table=nn.Embedding(vocab_size,vocab_size)
    def forward(self, xb,yb):
        logits=self.token_embedding_table(xb)
        B,C,T=logits.shape
        loss=F.cross_entropy(logits.reshape(B*C,T),yb.reshape(B*C))
        return logits,loss
m=BigramLanguageModel(len(voc))
logit,loss=m(xb,yb)
print(logit.shape)
print(loss)
        
print(len(voc))

ValueError: Expected input batch_size (32) to match target batch_size (4).